In [32]:
import numpy as np
import pandas as pd
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader

class TimeSeriesDataset(Dataset):
    def __init__(self, df, input_window=14, output_window=14, target_col_name='transaction_count'):
        """
        df               : A pandas DataFrame that includes both features and the target column.
        input_window     : Number of time steps used as input (lookback).
        output_window    : Number of time steps to forecast (e.g. 14 for 14-day).
        target_col_name  : Name of the target column in df.
        """
        # Separate the features vs. target
        self.X_data = df.values  # all columns except target
        self.y_data = df[target_col_name].values               

        self.input_window = input_window
        self.output_window = output_window

    def __len__(self):
        # The maximum valid index is total_rows - (input_window + output_window)
        return len(self.X_data) - (self.input_window + self.output_window - 1)

    def __getitem__(self, idx):
        # 1) Slice out the input window
        x_start = idx
        x_end   = idx + self.input_window
        X = self.X_data[x_start : x_end]   # shape => (input_window, num_features)

        # 2) Slice out the next 'output_window' points of the target
        y_start = x_end
        y_end   = x_end + self.output_window
        Y = self.y_data[y_start : y_end]   # shape => (output_window,)

        # Convert to float32 for PyTorch
        X = X.astype(np.float32)
        Y = Y.astype(np.float32)
        return X, Y


In [33]:
class TCNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, dilation=1):
        super().__init__()
        padding = (kernel_size - 1) * dilation
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, 
                              stride=1, padding=padding, dilation=dilation)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2) 
        
    def forward(self, x):
        """
        x shape: [batch_size, channels, seq_len]
        """
        out = self.conv(x)
        # Remove extra time-steps from padding to maintain causality
        out = out[:, :, :-self.conv.padding[0]]  # remove the last "padding" points
        out = self.relu(out)
        out = self.dropout(out)
        return out
    
class TCN(nn.Module):
    def __init__(self, in_channels, channel_list, kernel_size=3, dropout=0.2):
        super().__init__()
        blocks = []
        current_in = in_channels
        for i, out_ch in enumerate(channel_list):
            dilation = 2 ** i
            block = TCNBlock(current_in, out_ch, kernel_size, dilation=dilation)
            blocks.append(block)
            current_in = out_ch
        self.network = nn.Sequential(*blocks)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.network(x)
        x = self.dropout(x)
        return x

    
class TCNForecastingModel(nn.Module):
    def __init__(self, num_inputs, num_channels, output_size, kernel_size=2, dropout=0.2, input_window=30):
        super(TCNForecastingModel, self).__init__()
        self.tcn = TCN(num_inputs, num_channels, kernel_size, dropout)
        self.fc = nn.Linear(num_channels[-1], output_size)
        self.input_window = input_window

    def forward(self, x):
        x = x.permute(0, 2, 1)
        y = self.tcn(x)
        y = y[:, :, -1]
        return self.fc(y)

In [34]:
# Load data
import os

os.chdir("/Users/megan/Thesis/")
print("Current working directory:", os.getcwd())
data = pd.read_csv('data/top_800_product_groups.csv')
data['date'] = pd.to_datetime(data['date'], errors='coerce')

data["day_of_week"] = data["date"].dt.dayofweek 
data["day_of_month"] = data["date"].dt.day
data["month"] = data["date"].dt.month
data["day_of_year"] = data["date"].dt.dayofyear
data["year"] = data["date"].dt.year

# Define numerical columns (exclude 'product_group' and 'date')
numerical_columns = [
    'transaction_count',
    'avg_price',
    'unique_customers',
    'unique_articles_sold',
    'median_age',
    'fashion_news_subscribers',
    'first_purchase_days_ago',
    'recent_purchase_days_ago',
    'day_of_week',
    'day_of_month',
    'month',
    'day_of_year',
    'year'
]

categorical_columns = [
    'sales_channel_2',
    'most_common_age_bin_20-29',
    'most_common_age_bin_30-39',
    'most_common_age_bin_40-49',
    'most_common_age_bin_50-59',
    'most_common_age_bin_60+'
]

Current working directory: /Users/megan/Thesis


In [35]:
categorical_columns = [
    'sales_channel_2', 'most_common_age_bin_20-29', 'most_common_age_bin_30-39',
    'most_common_age_bin_40-49', 'most_common_age_bin_50-59', 'most_common_age_bin_60+'
]

# One-hot encode the categorical columns
data = pd.get_dummies(data, columns=categorical_columns, dtype=int)

data.drop(columns=['std_price', 'club_member_ratio'], inplace=True)



In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CausalConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation=1):
        """
        A 1D convolution that pads only on the left to ensure causality.
        """
        super(CausalConv1d, self).__init__()
        self.kernel_size = kernel_size
        self.dilation = dilation
        # No padding here; we will pad manually in forward.
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size,
                              padding=0, dilation=dilation)
        
    def forward(self, x):
        # Calculate the required left padding: (kernel_size-1) * dilation
        pad = (self.kernel_size - 1) * self.dilation
        # Pad only on the left: (left, right)
        x = F.pad(x, (pad, 0))
        return self.conv(x)

class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation, dropout=0.2):
        super(TemporalBlock, self).__init__()
        # Use CausalConv1d instead of regular Conv1d with symmetric padding.
        self.conv1 = CausalConv1d(in_channels, out_channels, kernel_size, dilation=dilation)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)
        
        self.conv2 = CausalConv1d(out_channels, out_channels, kernel_size, dilation=dilation)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)
        
        self.net = nn.Sequential(
            self.conv1, self.relu1, self.dropout1,
            self.conv2, self.relu2, self.dropout2
        )
        self.downsample = nn.Conv1d(in_channels, out_channels, 1) \
                          if in_channels != out_channels else None
        self.relu = nn.ReLU()
        
    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

In [37]:
top_10_groups = [
    "Bra Black Solid",
    "Dress Black Solid",
    "Leggings/Tights Black Solid",
    "Sweater Black Solid",
    "T-shirt Black Solid",
    "T-shirt White Solid",
    "Top Black Solid",
    "Trousers Black Solid",
    "Trousers Blue Denim",
    "Vest top Black Solid"
]

# remove top 10 products from pretrain data
pretrain_data = data[~data['product_group'].isin(top_10_groups)].copy()
top10_data = data[data['product_group'].isin(top_10_groups)].copy()

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader

pretrain_data = pretrain_data.sort_values(["date", "product_group"]).drop(columns=['date', 'product_group'])

# Take out the last 20% of data (never used in training or validation)
total_samples = len(pretrain_data)
leave_out_size = int(total_samples * 0.2)  # 20% to leave out
usable_data = pretrain_data.iloc[:-leave_out_size]  # Keep 80% for pretraining

# Split remaining 80% into 80% train, 20% validation
split_idx = int(len(usable_data) * 0.8)
train_data = usable_data.iloc[:split_idx]
val_data = usable_data.iloc[split_idx:]

feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

X_train_scaled = feature_scaler.fit_transform(train_data.drop(columns=['transaction_count']))
y_train_scaled = target_scaler.fit_transform(train_data[['transaction_count']])

X_val_scaled = feature_scaler.transform(val_data.drop(columns=['transaction_count']))
y_val_scaled = target_scaler.transform(val_data[['transaction_count']])

# Create scaled DataFrames
train_df_scaled = pd.DataFrame(X_train_scaled, columns=train_data.columns.drop('transaction_count'))
train_df_scaled['transaction_count'] = y_train_scaled

val_df_scaled = pd.DataFrame(X_val_scaled, columns=val_data.columns.drop('transaction_count'))
val_df_scaled['transaction_count'] = y_val_scaled

train_dataset = TimeSeriesDataset(train_df_scaled, input_window=14, output_window=1, target_col_name='transaction_count')
val_dataset = TimeSeriesDataset(val_df_scaled, input_window=14, output_window=1, target_col_name='transaction_count')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"Train size: {len(train_df_scaled)}, Validation size: {len(val_df_scaled)}, Left-out test size: {leave_out_size}")


Train size: 337648, Validation size: 84412, Left-out test size: 105515


In [39]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Subset
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from modules.utils import process_name

all_groups_results = {}
output_dir = "final_version/output/1_day/transfer_finetuned"
os.makedirs(output_dir, exist_ok=True)

for product_group in top_10_groups:
    print(f"\n=== Running for {product_group} ===")
    
    # Create a folder for the product group
    sanitized_group = process_name(product_group)
    group_output_dir = os.path.join(output_dir, sanitized_group)
    os.makedirs(group_output_dir, exist_ok=True)
    
    # 1. Filter and preprocess data for the product group
    # (Using the top10_data split defined earlier; here we filter by product_group)
    group_df = data[data['product_group'] == product_group].copy()
    # Optionally, if you want to use dates for plotting, you could keep the 'date' column.
    # For now, we drop it (along with 'product_group') as before.
    group_df = group_df.drop(columns=['date', 'product_group'])
    
    # Scale data (using the scalers fitted on pretrain_data)
    X_group = group_df.drop(columns=['transaction_count']).values
    y_group = group_df['transaction_count'].values.reshape(-1, 1)
    X_group_scaled = feature_scaler.transform(X_group)
    y_group_scaled = target_scaler.transform(y_group)
    
    group_df_scaled = pd.DataFrame(X_group_scaled, columns=[col for col in group_df.columns if col != 'transaction_count'])
    group_df_scaled['transaction_count'] = y_group_scaled
    group_df_scaled = group_df_scaled.sort_index()
    
    # Create the TimeSeriesDataset
    input_window, output_window = 14, 1
    target_col_name = 'transaction_count'
    group_dataset = TimeSeriesDataset(
        group_df_scaled,
        input_window=input_window,
        output_window=output_window,
        target_col_name=target_col_name
    )
    
    # Use the last 20% of the samples as the test set
    split_idx = int(0.8 * len(group_dataset))
    test_subset = Subset(group_dataset, list(range(split_idx, len(group_dataset))))
    test_loader = DataLoader(test_subset, batch_size=32, shuffle=False)
    
    # Initialize the model and load the pretrained weights
    num_features = group_df_scaled.shape[1]
    model = TCNForecastingModel(
        num_inputs=num_features,
        num_channels=[60, 60],
        output_size=1,
        kernel_size=5,
        dropout=0.3362
    )
    checkpoint = torch.load("fine_tuned_tcn.pt")
    model.load_state_dict(checkpoint, strict=False)
    
    # Evaluate the pretrained model on the test set
    model.eval()
    all_preds, all_targets = [], []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            preds = model(X_batch)
            all_preds.append(preds.cpu().numpy())
            all_targets.append(y_batch.cpu().numpy())
    all_preds = np.concatenate(all_preds, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)
    
    # Compute evaluation metrics
    mae = mean_absolute_error(all_targets, all_preds)
    rmse = np.sqrt(mean_squared_error(all_targets, all_preds))
    mape = np.mean(np.abs((all_targets - all_preds) / np.maximum(np.abs(all_targets), 1e-8))) * 100
    r2 = r2_score(all_targets, all_preds)
    
    print(f"Test Set for {product_group} => MAE: {mae:.4f}, RMSE: {rmse:.4f}, MAPE: {mape:.2f}%, R²: {r2:.4f}")
    
    # Plot Predictions vs Actual for the entire test set
    time_steps = np.arange(len(all_targets))
    plt.figure(figsize=(12, 6))
    plt.plot(time_steps, all_targets[:, 0], label="Actual")
    plt.plot(time_steps, all_preds[:, 0], label="Predicted", linestyle='--')
    plt.xlabel("Time Step")
    plt.ylabel("Scaled Transaction Count")
    plt.title(f"Transfer Learning with TCN Model for {product_group} (1-Day Predictions)")
    plt.legend()
    pred_fig_path = os.path.join(group_output_dir, "predictions_vs_actual.png")
    plt.savefig(pred_fig_path)
    plt.close()
    
    # Plot Residuals for the entire test set
    residuals = all_targets[:, 0] - all_preds[:, 0]
    plt.figure(figsize=(12, 6))
    plt.plot(time_steps, residuals, label="Residuals", color='green')
    plt.axhline(0, color='red', linestyle='--', alpha=0.7)
    plt.xlabel("Time Step")
    plt.ylabel("Residual")
    plt.title(f"Transfer Learning for {product_group} (1-Day)")
    plt.legend()
    resid_fig_path = os.path.join(group_output_dir, "prediction_residuals.png")
    plt.savefig(resid_fig_path)
    plt.close()
    
    # Save test set metrics
    metrics_dict = {'MAE': mae, 'RMSE': rmse, 'MAPE': mape, 'R2': r2}
    metrics_df = pd.DataFrame([metrics_dict])
    metrics_df.to_csv(os.path.join(group_output_dir, "test_metrics.csv"), index=False)
    
    # Store results
    all_groups_results[product_group] = {'metrics': metrics_dict}

print("\nAll groups processed. Figures and metrics saved in:", output_dir)



=== Running for Bra Black Solid ===
Test Set for Bra Black Solid => MAE: 0.1770, RMSE: 0.1868, MAPE: 93.28%, R²: -8.8590


/Users/megan/Thesis/thesis_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Users/megan/Thesis/thesis_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/var/folders/33/sqxmhw8j72g_q70vx5jqs_jr0000gn/T/ipykernel_24770/2796880866.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Ar


=== Running for Dress Black Solid ===
Test Set for Dress Black Solid => MAE: 0.3409, RMSE: 0.3786, MAPE: 96.31%, R²: -4.2867


/Users/megan/Thesis/thesis_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Users/megan/Thesis/thesis_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/var/folders/33/sqxmhw8j72g_q70vx5jqs_jr0000gn/T/ipykernel_24770/2796880866.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Ar


=== Running for Leggings/Tights Black Solid ===
Test Set for Leggings/Tights Black Solid => MAE: 0.1167, RMSE: 0.1266, MAPE: 83.95%, R²: -5.4521


...


=== Running for Trousers Blue Denim ===
Test Set for Trousers Blue Denim => MAE: 0.1632, RMSE: 0.1732, MAPE: 92.80%, R²: -7.8939


/Users/megan/Thesis/thesis_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Users/megan/Thesis/thesis_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/var/folders/33/sqxmhw8j72g_q70vx5jqs_jr0000gn/T/ipykernel_24770/2796880866.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Ar


=== Running for Vest top Black Solid ===
Test Set for Vest top Black Solid => MAE: 0.2149, RMSE: 0.2359, MAPE: 92.87%, R²: -5.0461


/Users/megan/Thesis/thesis_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Users/megan/Thesis/thesis_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/var/folders/33/sqxmhw8j72g_q70vx5jqs_jr0000gn/T/ipykernel_24770/2796880866.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Ar


All groups processed. Figures and metrics saved in: final_version/output/1_day/transfer_finetuned


In [40]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Subset
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import optuna

from modules.utils import process_name

############################
# Optuna Objective Function
############################
def objective_fn(trial, group_df_scaled, pretrained_checkpoint):
    """
    Creates a new TimeSeriesDataset with a trial-suggested input_window,
    splits it into train/val (80/20), and trains a TCN model with trial-suggested
    hyperparameters. Returns the validation RMSE.
    """
    # === Sample hyperparameters ===
    input_window = trial.suggest_int("input_window", 7, 28, step=7)
    lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
    dropout = trial.suggest_float("dropout", 0.0, 0.5)
    epochs = trial.suggest_int("epochs", 30, 80)

    # === Build TimeSeriesDataset with the chosen input_window ===
    dataset = TimeSeriesDataset(
        df=group_df_scaled,
        input_window=input_window,
        output_window=1,
        target_col_name="transaction_count"
    )

    # === Split into train/val (80/20) ===
    total_samples = len(dataset)
    train_size = int(0.8 * total_samples)
    train_indices = range(train_size)
    val_indices = range(train_size, total_samples)

    train_loader = DataLoader(Subset(dataset, train_indices), batch_size=32, shuffle=False)
    val_loader = DataLoader(Subset(dataset, val_indices), batch_size=32, shuffle=False)

    # === Build model (fixed architecture except dropout) ===
    #    If you must keep the same number of channels as the pretrained model, do so here:
    model = TCNForecastingModel(
        num_inputs=group_df_scaled.shape[1],
        num_channels=[64, 64],  # Must match pretrained if you want to load all weights
        output_size=1,
        kernel_size=3,
        dropout=dropout
    )

    # === Load pretrained weights ===
    checkpoint = torch.load(pretrained_checkpoint)
    model.load_state_dict(checkpoint, strict=False)

    # === Fine-tune model on train split ===
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    model.train()
    for epoch in range(epochs):
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            preds = model(X_batch)
            loss = criterion(preds, y_batch)
            loss.backward()
            optimizer.step()

    # === Evaluate on validation split ===
    model.eval()
    all_val_preds, all_val_targets = [], []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            preds = model(X_batch)
            all_val_preds.append(preds.cpu().numpy())
            all_val_targets.append(y_batch.cpu().numpy())

    all_val_preds = np.concatenate(all_val_preds, axis=0)
    all_val_targets = np.concatenate(all_val_targets, axis=0)

    val_rmse = np.sqrt(mean_squared_error(all_val_targets, all_val_preds))
    return val_rmse


############################
# Main Loop
############################
all_groups_results = {}
output_dir = "final_version/output/1_day/transfer"
os.makedirs(output_dir, exist_ok=True)

for product_group in top_10_groups:
    print(f"\n=== Running for {product_group} ===")
    
    # Create output folder
    sanitized_group = process_name(product_group)
    group_output_dir = os.path.join(output_dir, sanitized_group)
    os.makedirs(group_output_dir, exist_ok=True)
    
    # === Load and scale data for this product group ===
    group_df = data[data['product_group'] == product_group].copy()
    group_df = group_df.drop(columns=['date', 'product_group'])
    
    X_group = group_df.drop(columns=['transaction_count']).values
    y_group = group_df['transaction_count'].values.reshape(-1, 1)
    
    X_group_scaled = feature_scaler.transform(X_group)
    y_group_scaled = target_scaler.transform(y_group)
    
    group_df_scaled = pd.DataFrame(
        X_group_scaled,
        columns=[col for col in group_df.columns if col != 'transaction_count']
    )
    group_df_scaled['transaction_count'] = y_group_scaled
    group_df_scaled = group_df_scaled.sort_index()
    
    # === Optuna hyperparameter search (including input_window) ===
    study = optuna.create_study(direction="minimize")
    study.optimize(
        lambda trial: objective_fn(
            trial=trial,
            group_df_scaled=group_df_scaled,
            pretrained_checkpoint="pretrained_tcn_1_day.pt"
        ),
        n_trials=30
    )
    
    best_params = study.best_trial.params
    print(f"Best parameters for {product_group}: {best_params}")
    
    # === Rebuild final dataset with best input_window ===
    final_input_window = best_params["input_window"]
    final_dataset = TimeSeriesDataset(
        df=group_df_scaled,
        input_window=final_input_window,
        output_window=1,
        target_col_name="transaction_count"
    )
    
    # === Final train/test split (80/20) ===
    total_samples = len(final_dataset)
    test_split = int(0.8 * total_samples)
    final_train_indices = range(test_split)
    final_test_indices = range(test_split, total_samples)

    final_train_loader = DataLoader(Subset(final_dataset, final_train_indices), batch_size=32, shuffle=False)
    final_test_loader = DataLoader(Subset(final_dataset, final_test_indices), batch_size=32, shuffle=False)

    # === Build final model using best hyperparams ===
    model_final = TCNForecastingModel(
        num_inputs=group_df_scaled.shape[1],
        num_channels=[64, 64],  # Must match pretrained
        output_size=1,
        kernel_size=3,
        dropout=best_params["dropout"]
    )
    checkpoint = torch.load("pretrained_tcn_1_day.pt")
    model_final.load_state_dict(checkpoint, strict=False)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model_final.parameters(), lr=best_params["lr"])
    final_epochs = best_params["epochs"]
    
    # === Final training on 80% data ===
    model_final.train()
    for epoch in range(final_epochs):
        epoch_loss = 0.0
        for X_batch, y_batch in final_train_loader:
            optimizer.zero_grad()
            preds = model_final(X_batch)
            loss = criterion(preds, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        avg_loss = epoch_loss / len(final_train_loader)
        print(f"Epoch {epoch+1}/{final_epochs}, Loss: {avg_loss:.6f}")
    
    # === Evaluate final model on the test set ===
    model_final.eval()
    all_preds, all_targets = [], []
    with torch.no_grad():
        for X_batch, y_batch in final_test_loader:
            preds = model_final(X_batch)
            all_preds.append(preds.cpu().numpy())
            all_targets.append(y_batch.cpu().numpy())
  
    all_preds = np.concatenate(all_preds, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)

    all_preds_unscaled = target_scaler.inverse_transform(all_preds)
    all_targets_unscaled = target_scaler.inverse_transform(all_targets)

    mae = mean_absolute_error(all_targets_unscaled, all_preds_unscaled)
    rmse = np.sqrt(mean_squared_error(all_targets_unscaled, all_preds_unscaled))
    mape = np.mean(np.abs((all_targets_unscaled - all_preds_unscaled) /
                        np.maximum(np.abs(all_targets_unscaled), 1e-8))) * 100
    r2 = r2_score(all_targets_unscaled, all_preds_unscaled)

    print(f"Final Test for {product_group} => MAE: {mae:.4f}, RMSE: {rmse:.4f}, MAPE: {mape:.2f}%, R²: {r2:.4f}")
    
    # === Plot Predictions vs. Actual ===
    time_steps = np.arange(len(all_targets))
    plt.figure(figsize=(12, 6))
    plt.plot(time_steps, all_targets[:, 0], label="Actual", color='blue')
    plt.plot(time_steps, all_preds[:, 0], label="Predicted", color='red', linestyle='--')
    plt.xlabel("Time Step")
    plt.ylabel("Scaled Transaction Count")
    plt.title(f"Final TCN Model for {product_group} (Input Window={final_input_window})")
    plt.legend()
    plt.savefig(os.path.join(group_output_dir, "predictions_vs_actual.png"))
    plt.close()
    
    # === Plot Residuals ===
    residuals = all_targets[:, 0] - all_preds[:, 0]
    plt.figure(figsize=(12, 6))
    plt.plot(time_steps, residuals, label="Residuals", color='green')
    plt.axhline(0, color='red', linestyle='--', alpha=0.7)
    plt.xlabel("Time Step")
    plt.ylabel("Residual")
    plt.title(f"{product_group}: Residuals (Test Set)")
    plt.legend()
    plt.savefig(os.path.join(group_output_dir, "prediction_residuals.png"))
    plt.close()
    
    # === Save final metrics ===
    metrics_dict = {
        'MAE': mae,
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2
    }
    metrics_df = pd.DataFrame([metrics_dict])
    metrics_df.to_csv(os.path.join(group_output_dir, "test_metrics.csv"), index=False)
    
    # === Store results in dictionary ===
    all_groups_results[product_group] = {'best_params': best_params, 'metrics': metrics_dict}

print("\nAll groups processed. Results saved in:", output_dir)


/Users/megan/Thesis/thesis_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Users/megan/Thesis/thesis_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
[I 2025-03-05 13:35:16,490] A new study created in memory with name: no-name-de3b8f31-8c34-4dfd-87cd-f46c6bb99bec



=== Running for Bra Black Solid ===


/var/folders/33/sqxmhw8j72g_q70vx5jqs_jr0000gn/T/ipykernel_24770/1428423725.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(pretrained_checkpoin

Best parameters for Bra Black Solid: {'input_window': 28, 'lr': 1.0693890573918646e-05, 'dropout': 0.35990957946073043, 'epochs': 37}
Epoch 1/37, Loss: 0.006980
Epoch 2/37, Loss: 0.007139
Epoch 3/37, Loss: 0.006669
Epoch 4/37, Loss: 0.006584
Epoch 5/37, Loss: 0.006034
Epoch 6/37, Loss: 0.006031
Epoch 7/37, Loss: 0.005941
Epoch 8/37, Loss: 0.006055
Epoch 9/37, Loss: 0.005689
Epoch 10/37, Loss: 0.005559
Epoch 11/37, Loss: 0.005611
Epoch 12/37, Loss: 0.006407
Epoch 13/37, Loss: 0.006533
Epoch 14/37, Loss: 0.005916
Epoch 15/37, Loss: 0.005904
Epoch 16/37, Loss: 0.005786
Epoch 17/37, Loss: 0.005985
Epoch 18/37, Loss: 0.005927
Epoch 19/37, Loss: 0.005989
Epoch 20/37, Loss: 0.005729
Epoch 21/37, Loss: 0.005860
Epoch 22/37, Loss: 0.005702
Epoch 23/37, Loss: 0.005871
Epoch 24/37, Loss: 0.005771
Epoch 25/37, Loss: 0.005811
Epoch 26/37, Loss: 0.006022
Epoch 27/37, Loss: 0.005784
Epoch 28/37, Loss: 0.006083
Epoch 29/37, Loss: 0.005960
Epoch 30/37, Loss: 0.005868
Epoch 31/37, Loss: 0.006032
Epoch 3

/Users/megan/Thesis/thesis_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Users/megan/Thesis/thesis_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
[I 2025-03-05 13:40:06,403] A new study created in memory with name: no-name-a0ca083e-d2c1-4ecd-b53f-b646f6b9a53e


...

Best parameters for Trousers Blue Denim: {'input_window': 14, 'lr': 3.6613061313532705e-05, 'dropout': 0.20073745708336618, 'epochs': 55}
Epoch 1/55, Loss: 0.008323
Epoch 2/55, Loss: 0.007346
Epoch 3/55, Loss: 0.007134
Epoch 4/55, Loss: 0.007703
Epoch 5/55, Loss: 0.007024
Epoch 6/55, Loss: 0.007075
Epoch 7/55, Loss: 0.007647
Epoch 8/55, Loss: 0.007380
Epoch 9/55, Loss: 0.007604
Epoch 10/55, Loss: 0.007440
Epoch 11/55, Loss: 0.006795
Epoch 12/55, Loss: 0.007005
Epoch 13/55, Loss: 0.007034
Epoch 14/55, Loss: 0.007062
Epoch 15/55, Loss: 0.007059
Epoch 16/55, Loss: 0.007195
Epoch 17/55, Loss: 0.006679
Epoch 18/55, Loss: 0.006759
Epoch 19/55, Loss: 0.007358
Epoch 20/55, Loss: 0.006790
Epoch 21/55, Loss: 0.006811
Epoch 22/55, Loss: 0.006753
Epoch 23/55, Loss: 0.006788
Epoch 24/55, Loss: 0.006638
Epoch 25/55, Loss: 0.006829
Epoch 26/55, Loss: 0.007086
Epoch 27/55, Loss: 0.007020
Epoch 28/55, Loss: 0.007055
Epoch 29/55, Loss: 0.006683
Epoch 30/55, Loss: 0.007038
Epoch 31/55, Loss: 0.006857
Epo

/Users/megan/Thesis/thesis_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Users/megan/Thesis/thesis_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
[I 2025-03-05 14:19:45,075] A new study created in memory with name: no-name-24646c62-5be8-4344-9c38-541eb26e5805



=== Running for Vest top Black Solid ===


/var/folders/33/sqxmhw8j72g_q70vx5jqs_jr0000gn/T/ipykernel_24770/1428423725.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(pretrained_checkpoin

Best parameters for Vest top Black Solid: {'input_window': 28, 'lr': 0.000939251547608988, 'dropout': 0.34552490639340255, 'epochs': 40}
Epoch 1/40, Loss: 0.009715
Epoch 2/40, Loss: 0.008887
Epoch 3/40, Loss: 0.009651
Epoch 4/40, Loss: 0.011985
Epoch 5/40, Loss: 0.010953
Epoch 6/40, Loss: 0.010921
Epoch 7/40, Loss: 0.008547
Epoch 8/40, Loss: 0.008256
Epoch 9/40, Loss: 0.007553
Epoch 10/40, Loss: 0.007244
Epoch 11/40, Loss: 0.007316
Epoch 12/40, Loss: 0.007665
Epoch 13/40, Loss: 0.006481
Epoch 14/40, Loss: 0.008027
Epoch 15/40, Loss: 0.006734
Epoch 16/40, Loss: 0.008280
Epoch 17/40, Loss: 0.007192
Epoch 18/40, Loss: 0.007503
Epoch 19/40, Loss: 0.006971
Epoch 20/40, Loss: 0.006878
Epoch 21/40, Loss: 0.006849
Epoch 22/40, Loss: 0.007219
Epoch 23/40, Loss: 0.008011
Epoch 24/40, Loss: 0.006397
Epoch 25/40, Loss: 0.006294
Epoch 26/40, Loss: 0.006454
Epoch 27/40, Loss: 0.006102
Epoch 28/40, Loss: 0.005736
Epoch 29/40, Loss: 0.006296
Epoch 30/40, Loss: 0.006171
Epoch 31/40, Loss: 0.005913
Epoc

In [42]:
import os
import pandas as pd

output_dir = "final_version/output/1_day/transfer"

metrics_summary = []

# Gather metrics from each product group's folder
for product_group in top_10_groups:
    sanitized_group = process_name(product_group)
    group_output_dir = os.path.join(output_dir, sanitized_group)
    metrics_file = os.path.join(group_output_dir, "test_metrics.csv")
    
    if os.path.exists(metrics_file):
        df = pd.read_csv(metrics_file)
        # Assume test_metrics.csv has columns: MAE, RMSE, MAPE, R2
        # and just one row of metrics
        row = df.iloc[0].to_dict()
        row["Product Group"] = product_group
        metrics_summary.append(row)
    else:
        print(f"Warning: No metrics file found for {product_group}")

# Convert to a DataFrame
summary_df = pd.DataFrame(metrics_summary)

# Reorder columns for clarity
cols_order = ["Product Group", "MAE", "RMSE", "MAPE", "R2"]
summary_df = summary_df[cols_order]

# Save the summary of all product groups
summary_file = os.path.join(output_dir, "final_metrics_summary.csv")
summary_df.to_csv(summary_file, index=False)

# Compute and save average metrics across all products
avg_metrics = summary_df[["MAE", "RMSE", "MAPE", "R2"]].mean()
avg_metrics_df = pd.DataFrame([avg_metrics])
avg_metrics_file = os.path.join(output_dir, "final_test_avg_metrics.csv")
avg_metrics_df.to_csv(avg_metrics_file, index=False)

print(f"Summary metrics saved to: {summary_file}")
print(f"Average metrics saved to: {avg_metrics_file}")


Summary metrics saved to: final_version/output/1_day/transfer/final_metrics_summary.csv
Average metrics saved to: final_version/output/1_day/transfer/final_test_avg_metrics.csv


## Macroeconomic Indicators

In [44]:
consumer_sentiment = pd.read_csv('data/external/consumer_sentiment.csv')
consumer_sentiment['DATE'] = pd.to_datetime(consumer_sentiment['DATE'])
consumer_sentiment.set_index('DATE', inplace=True)
consumer_sentiment = consumer_sentiment.resample('D').ffill()

cpi = pd.read_csv('data/external/cpi_data.csv')
cpi['DATE'] = pd.to_datetime(cpi['DATE'])
cpi.set_index('DATE', inplace=True)
cpi = cpi.resample('D').ffill()

gdp = pd.read_csv('data/external/gdp_data.csv')
gdp['DATE'] = pd.to_datetime(gdp['DATE'])
gdp.set_index('DATE', inplace=True)
gdp = gdp.resample('D').ffill()

unemployment = pd.read_csv('data/external/unemployment_data.csv')
unemployment['DATE'] = pd.to_datetime(unemployment['DATE'])
unemployment.set_index('DATE', inplace=True)
unemployment = unemployment.resample('D').ffill()

In [45]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from modules.utils import process_name

# Output directory
output_dir = "final_version/output/macroeconomic/unemployment/tcn_v2"
os.makedirs(output_dir, exist_ok=True)

all_groups_results = {}

for product_group in top_10_groups:
    print(f"\n=== Processing {product_group} ===")
    sanitized_group = process_name(product_group)
    group_output_dir = os.path.join(output_dir, sanitized_group)
    os.makedirs(group_output_dir, exist_ok=True)

    # 1. Filter data for this product group
    product_data = data[data['product_group'] == product_group].copy()
    if product_data.empty:
        print(f"No data for {product_group}, skipping.")
        continue

    # Ensure date is the index
    product_data['date'] = pd.to_datetime(product_data['date'], errors='coerce')
    product_data.set_index('date', inplace=True)
    product_data = product_data.asfreq('D').fillna(0)

    # 2. Merge Macroeconomic Data 
    print(f"🔄 Merging macroeconomic data for {product_group}...")
    product_data = product_data.join(unemployment[['UNRATE']], how='left')
    product_data.fillna(method='ffill', inplace=True)  # Forward fill missing values

    # 3. Drop unwanted columns
    drop_cols = ['product_group', 'product_type_name', 'colour_group_name', 'graphical_appearance_name']
    product_data.drop(columns=[c for c in drop_cols if c in product_data.columns], inplace=True, errors='ignore')

    if 'transaction_count' not in product_data.columns:
        print(f"No transaction_count in {product_group}, skipping.")
        continue

    # 4. Train/Validation Split (80/20)
    total_len = len(product_data)
    if total_len < 50:
        print(f"Not enough data for {product_group}, skipping.")
        continue
    split_idx = int(0.8 * total_len)
    train_df = product_data.iloc[:split_idx].copy()
    val_df = product_data.iloc[split_idx:].copy()

    # 5. Scale features & target
    feature_scaler = MinMaxScaler()
    target_scaler = MinMaxScaler()

    X_train = train_df.drop(columns=['transaction_count'])
    y_train = train_df[['transaction_count']]

    X_val = val_df.drop(columns=['transaction_count'])
    y_val = val_df[['transaction_count']]

    X_train_scaled = feature_scaler.fit_transform(X_train)
    y_train_scaled = target_scaler.fit_transform(y_train)

    X_val_scaled = feature_scaler.transform(X_val)
    y_val_scaled = target_scaler.transform(y_val)

    train_df_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
    train_df_scaled['transaction_count'] = y_train_scaled

    val_df_scaled = pd.DataFrame(X_val_scaled, columns=X_val.columns, index=X_val.index)
    val_df_scaled['transaction_count'] = y_val_scaled

    # 6. Create TimeSeriesDataset
    input_window, output_window = 14, 1
    train_dataset = TimeSeriesDataset(
        train_df_scaled,
        input_window=input_window,
        output_window=output_window,
        target_col_name='transaction_count'
    )
    val_dataset = TimeSeriesDataset(
        val_df_scaled,
        input_window=input_window,
        output_window=output_window,
        target_col_name='transaction_count'
    )

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    # 7. Build TCN
    num_features = X_train.shape[1] + 1 
    model = TCNForecastingModel(
        num_inputs=num_features,
        num_channels=[64, 64],
        output_size=1,
        kernel_size=3,
        dropout=0.2
    )

    # 8. Train TCN
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    epochs = 50

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            preds = model(X_batch)
            loss = criterion(preds, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                preds = model(X_batch)
                loss = criterion(preds, y_batch)
                val_loss += loss.item()
        val_loss /= len(val_loader)

        if (epoch + 1) % 10 == 0:
            print(f"{product_group} | Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")

    # 9. Evaluate on Validation
    model.eval()
    all_preds, all_targets = [], []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            preds = model(X_batch)
            all_preds.append(preds.cpu().numpy())
            all_targets.append(y_batch.cpu().numpy())

    all_preds = np.concatenate(all_preds, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)

    # Invert scaling
    preds_unscaled = target_scaler.inverse_transform(all_preds)
    targets_unscaled = target_scaler.inverse_transform(all_targets)

    mae = mean_absolute_error(targets_unscaled, preds_unscaled)
    rmse = np.sqrt(mean_squared_error(targets_unscaled, preds_unscaled))
    mape = np.mean(np.abs((targets_unscaled - preds_unscaled) / np.maximum(targets_unscaled, 1))) * 100
    r2 = r2_score(targets_unscaled, preds_unscaled)

    all_groups_results[product_group] = {'MAE': mae, 'RMSE': rmse, 'MAPE': mape, 'R2': r2}

    print(f"{product_group} => MAE: {mae:.4f}, RMSE: {rmse:.4f}, MAPE: {mape:.2f}%, R2: {r2:.4f}")

# Save summary
summary_df = pd.DataFrame.from_dict(all_groups_results, orient='index')
summary_df.to_csv(os.path.join(output_dir, "metrics_summary.csv"))

# Compute and save average metrics
avg_metrics = {
    'MAE': np.mean([all_groups_results[pg]['MAE'] for pg in all_groups_results]),
    'RMSE': np.mean([all_groups_results[pg]['RMSE'] for pg in all_groups_results]),
    'MAPE': np.mean([all_groups_results[pg]['MAPE'] for pg in all_groups_results]),
    'R2': np.mean([all_groups_results[pg]['R2'] for pg in all_groups_results])
}

avg_metrics_df = pd.DataFrame([avg_metrics])
avg_metrics_df.to_csv(os.path.join(output_dir, "avg_metrics.csv"), index=False)



=== Processing Bra Black Solid ===
🔄 Merging macroeconomic data for Bra Black Solid...


/var/folders/33/sqxmhw8j72g_q70vx5jqs_jr0000gn/T/ipykernel_24770/4261263564.py:39: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  product_data.fillna(method='ffill', inplace=True)  # Forward fill missing values


Bra Black Solid | Epoch 10/50, Train Loss: 0.007986, Val Loss: 0.004822
Bra Black Solid | Epoch 20/50, Train Loss: 0.007422, Val Loss: 0.004596
Bra Black Solid | Epoch 30/50, Train Loss: 0.006994, Val Loss: 0.004574
Bra Black Solid | Epoch 40/50, Train Loss: 0.006545, Val Loss: 0.004309
Bra Black Solid | Epoch 50/50, Train Loss: 0.006784, Val Loss: 0.004716
Bra Black Solid => MAE: 160.0898, RMSE: 197.7088, MAPE: 30.91%, R2: 0.0106

=== Processing Dress Black Solid ===
🔄 Merging macroeconomic data for Dress Black Solid...


/var/folders/33/sqxmhw8j72g_q70vx5jqs_jr0000gn/T/ipykernel_24770/4261263564.py:39: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  product_data.fillna(method='ffill', inplace=True)  # Forward fill missing values


Dress Black Solid | Epoch 10/50, Train Loss: 0.014203, Val Loss: 0.017184
Dress Black Solid | Epoch 20/50, Train Loss: 0.011841, Val Loss: 0.012281
Dress Black Solid | Epoch 30/50, Train Loss: 0.010475, Val Loss: 0.011497
Dress Black Solid | Epoch 40/50, Train Loss: 0.010078, Val Loss: 0.010855
Dress Black Solid | Epoch 50/50, Train Loss: 0.008548, Val Loss: 0.010640
Dress Black Solid => MAE: 310.4699, RMSE: 414.7816, MAPE: 30.38%, R2: 0.4351

=== Processing Leggings/Tights Black Solid ===
🔄 Merging macroeconomic data for Leggings/Tights Black Solid...


...

Trousers Black Solid | Epoch 10/50, Train Loss: 0.006704, Val Loss: 0.001506
Trousers Black Solid | Epoch 20/50, Train Loss: 0.006042, Val Loss: 0.001279
Trousers Black Solid | Epoch 30/50, Train Loss: 0.005831, Val Loss: 0.001232
Trousers Black Solid | Epoch 40/50, Train Loss: 0.005392, Val Loss: 0.001233
Trousers Black Solid | Epoch 50/50, Train Loss: 0.005319, Val Loss: 0.001216
Trousers Black Solid => MAE: 196.6200, RMSE: 256.5932, MAPE: 20.23%, R2: 0.1809

=== Processing Trousers Blue Denim ===
🔄 Merging macroeconomic data for Trousers Blue Denim...


/var/folders/33/sqxmhw8j72g_q70vx5jqs_jr0000gn/T/ipykernel_24770/4261263564.py:39: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  product_data.fillna(method='ffill', inplace=True)  # Forward fill missing values


Trousers Blue Denim | Epoch 10/50, Train Loss: 0.011492, Val Loss: 0.004454
Trousers Blue Denim | Epoch 20/50, Train Loss: 0.010883, Val Loss: 0.004202
Trousers Blue Denim | Epoch 30/50, Train Loss: 0.009409, Val Loss: 0.003788
Trousers Blue Denim | Epoch 40/50, Train Loss: 0.009416, Val Loss: 0.003429
Trousers Blue Denim | Epoch 50/50, Train Loss: 0.008136, Val Loss: 0.003276
Trousers Blue Denim => MAE: 134.2721, RMSE: 176.1062, MAPE: 30.39%, R2: 0.1860

=== Processing Vest top Black Solid ===
🔄 Merging macroeconomic data for Vest top Black Solid...


/var/folders/33/sqxmhw8j72g_q70vx5jqs_jr0000gn/T/ipykernel_24770/4261263564.py:39: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  product_data.fillna(method='ffill', inplace=True)  # Forward fill missing values


Vest top Black Solid | Epoch 10/50, Train Loss: 0.014740, Val Loss: 0.007566
Vest top Black Solid | Epoch 20/50, Train Loss: 0.012365, Val Loss: 0.005177
Vest top Black Solid | Epoch 30/50, Train Loss: 0.009953, Val Loss: 0.004577
Vest top Black Solid | Epoch 40/50, Train Loss: 0.009509, Val Loss: 0.003572
Vest top Black Solid | Epoch 50/50, Train Loss: 0.008509, Val Loss: 0.004121
Vest top Black Solid => MAE: 170.6302, RMSE: 214.2187, MAPE: 25.48%, R2: 0.5521
